In [3]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import TD3

In [4]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (0, 0, 0)
car_name = "TD3"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [15]:
folder = '/mnt/d/donkeycar_model/TD3/'

In [16]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [17]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
loading scene mini_monaco


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


In [18]:
for i in range(1000):
    if i == 0:
        model = TD3("CnnPolicy", env, n_steps=500, verbose=1, buffer_size=10000 )
        Log={"TestReward":[]}

    # training
    obs = model.set_env(env)
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_TD3-{(i+1)*2500}.npy", Log)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-3.0 False
-2.994835054888508 False
-2.9959999435906766 False
-2.935955752538078 False
-2.9132852813647405 False
-2.8912285353459692 False
-2.8544160772117695 False
-2.7459679853766112 False
-2.6679570296093726 False
-2.6406142432743813 False
-2.635615567920262 False
-2.5651772521931626 False
-2.50328601463232 False
-2.425151131907425 False
-2.3510559433678524 False
-2.312325798124226 False
-2.25124055121226 False
-2.2205475712153193 False
-2.155923386827283 False
-2.0445640017176565 False
-1.9998280482673438 False


KeyboardInterrupt: 

In [ ]:
num = 60000
test_model = PPO.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    # print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

In [9]:
env.close()